In [ ]:
# 使用github上面直接找到的库计算中文词语的相似度效果实在不是很好，这里自己手动利用专业领域的语料训练一个word2vec模型

In [ ]:
# 1.将text_data.txt和title_data.txt合并成一个文件，命名为data.txt
with open('./data/text_data.txt', 'r', encoding='utf-8') as f_1, open('./data/title_data.txt', 'r', encoding='utf-8') as f_2, open('./data/corups.txt', 'w', encoding='utf-8') as f_3:
    for line in f_1.readlines():
        f_3.write(line)
    for line in f_2.readlines():
        f_3.write(line)
print('合并完成')

In [2]:
# 2.将txt文件转化为csv文件
import csv

txt_file = './data/corpus.txt'
csv_file = './data/corpus.csv'


with open(txt_file, 'r') as input_file:
   
    with open(csv_file, 'w', newline='') as output_file:
        
        csv_writer = csv.writer(output_file)

        
        for line in input_file:
           
            fields = line.strip().split()

   
            csv_writer.writerow(fields)


In [3]:
# 3.语料预处理
import pandas as pd
import jieba    # 导入包
import re # 导入字符处理工具,正则表达式

def clean_text(data, stopword):
        """
        data: 数据，待分词的Series序列，用pandas读取
        stopword:停用词
        """
        # 去除无用字符
        pattern = re.compile(r'[\sA-Za-z～()（）【】%*#+-\.\\\/:=：__,，。、;；“”""''’‘？?！!<《》>^&{}|=……]')
        corpus_ = data.apply(lambda s: re.sub(pattern, '', str(s)))
        # 分词
        text = corpus_.apply(jieba.lcut)
        # 过滤通用词
        text = text.apply(lambda cut_words: [word for word in cut_words if word not in stopword])
        return text


In [4]:
# 加载语料
corpus = pd.read_csv("./data/corpus.csv", encoding='utf-8', error_bad_lines=False)
# 加载停用词
stop_word = [word for word in open("./data/stopwords.txt", 'r', encoding="utf-8")]


b'Skipping line 6868: expected 1 fields, saw 2\nSkipping line 6872: expected 1 fields, saw 2\nSkipping line 6875: expected 1 fields, saw 2\nSkipping line 6876: expected 1 fields, saw 2\nSkipping line 6879: expected 1 fields, saw 2\nSkipping line 6880: expected 1 fields, saw 2\nSkipping line 6881: expected 1 fields, saw 2\nSkipping line 6882: expected 1 fields, saw 2\nSkipping line 6883: expected 1 fields, saw 2\nSkipping line 6884: expected 1 fields, saw 2\nSkipping line 6885: expected 1 fields, saw 2\nSkipping line 6886: expected 1 fields, saw 2\nSkipping line 6887: expected 1 fields, saw 2\nSkipping line 6900: expected 1 fields, saw 2\nSkipping line 6901: expected 1 fields, saw 2\nSkipping line 6902: expected 1 fields, saw 2\nSkipping line 6903: expected 1 fields, saw 2\nSkipping line 6904: expected 1 fields, saw 2\nSkipping line 6905: expected 1 fields, saw 2\nSkipping line 6906: expected 1 fields, saw 2\nSkipping line 6907: expected 1 fields, saw 2\nSkipping line 6909: expected 1 f

In [16]:
# 开始分词并保存（重新保存为txt文件）
with open ('./data/corpus_cut.txt', 'w', encoding='utf-8') as f:
    for row in corpus.itertuples():
        # print(row[1])
        text = clean_text(pd.Series(row[1]), stop_word)
        # print(text[0])
        f.write(' '.join(text[0]) + '\n')
print('分词完成')


    

In [3]:
# 4.词向量训练
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
'''
LineSentence(open('data.txt', 'r', encoding='utf8'))：该部分从文件'data.txt'中读取文本数据，并将其准备为句子序列，其中每个句子是一个单词列表。LineSentence类用于逐行读取文件并将每行拆分为单词。

sg=0：该参数指定训练算法。这里，sg=0表示使用CBOW（Continuous Bag of Words）算法。如果sg=1，则表示使用Skip-gram算法。

size=100：该参数设置单词向量的维度。在这种情况下，每个单词将由一个大小为100的向量表示。

window=3：该参数确定上下文窗口大小。它指定目标单词与句子中上下文单词之间的最大距离。窗口大小为3意味着算法将考虑目标单词前面和后面的三个单词作为其上下文。

min_count=1：该参数设置单词在训练过程中的最小出现次数。出现次数小于min_count的单词将被忽略。

workers=8：该参数指定训练过程中要使用的线程数。它控制训练过程的并行化，并加快在多核系统上的训练速度。
'''

model = Word2Vec(
    LineSentence(open('./data/corpus_cut.txt', 'r', encoding='utf8')),
    sg = 0,
    vector_size = 100,
    window = 3,
    min_count = 1,
    workers=8
)
 
# 词向量保存
model.wv.save_word2vec_format('./data/corpus.vector', binary=False)
 
# 模型保存
model.save('./model/corpus.model')

In [3]:
# 词向量模型测试 
from gensim.models import Word2Vec
# 加载 .model 模型
model = Word2Vec.load("./model/corpus.model")

# 转储为二进制格式
model.wv.save_word2vec_format('./model/word2vec.bin', binary=True)


In [11]:
# 计算相似度
model = Word2Vec.load('./model/corpus.model')

word_vectors = model.wv

similarity_score = word_vectors.similarity('暴雨', '洪涝') # 注意：这里的词语必须在语料中出现过，否则会报错

print(similarity_score)

0.8775766


这里我们不直接使用这个训练好的词向量，我们将二进制的模型打包成gz格式的文件，替换掉synonyms原本的词向量文件（实测后发现这个效果甚至没有原来的词向量好...可能还是语料太小了）